In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torchvision
from torch.optim.lr_scheduler import StepLR

from model import EfficientNet_B0

from torchvision import transforms
from sklearn.metrics import average_precision_score
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
from voc_dataloader import VocDataset, VOC_CLASSES

%load_ext autoreload
%autoreload 2

In [2]:
batch = 128

transform = transforms.Compose(
    [transforms.RandomHorizontalFlip(),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch,
                                          shuffle=True, num_workers=4)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch,
                                         shuffle=False, num_workers=4)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

classifier = EfficientNet_B0().to(device)

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(classifier.parameters(), lr=0.001, momentum=0.9)
# optimizer = torch.optim.Adam(classifier.parameters(), lr=1e-4)

In [ ]:
for epoch in range(100):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = classifier(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 200 == 199:    # print every 100 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 199))
            running_loss = 0.0

print('Finished Training')

[1,   200] loss: 2.358
[2,   200] loss: 1.627
[3,   200] loss: 1.418
[4,   200] loss: 1.302
[5,   200] loss: 1.208
[6,   200] loss: 1.123
[7,   200] loss: 1.063
[8,   200] loss: 0.986
[9,   200] loss: 0.937
[10,   200] loss: 0.865
[11,   200] loss: 0.813
[12,   200] loss: 0.765
[13,   200] loss: 0.718
[14,   200] loss: 0.669
[15,   200] loss: 0.623
[16,   200] loss: 0.596
[17,   200] loss: 0.541
[18,   200] loss: 0.511
[19,   200] loss: 0.482
[20,   200] loss: 0.452
[21,   200] loss: 0.418
[22,   200] loss: 0.383
[23,   200] loss: 0.373
[24,   200] loss: 0.340
[25,   200] loss: 0.321
[26,   200] loss: 0.293
[27,   200] loss: 0.285
[28,   200] loss: 0.268
[29,   200] loss: 0.247
[30,   200] loss: 0.230
[31,   200] loss: 0.218
[32,   200] loss: 0.214
[33,   200] loss: 0.201
[34,   200] loss: 0.200
[35,   200] loss: 0.175
[36,   200] loss: 0.167
[37,   200] loss: 0.157
[38,   200] loss: 0.157
[39,   200] loss: 0.147
[40,   200] loss: 0.143
[41,   200] loss: 0.132
[42,   200] loss: 0.128
[

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image


def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images[0:4]))
# print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

In [ ]:
outputs = classifier(images.to(device))
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                              for j in range(4)))

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = classifier(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

In [ ]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = classifier(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

## Luke's pretrianed model